In [3]:
import requests
from random import choice
import json
from bs4 import BeautifulSoup

In [20]:
_user_agents = [
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/65.0.3325.181 Safari/537.36'
]
 
 
class InstagramScraper:
 
    def __init__(self, user_agents=None, proxy=None):
        self.user_agents = user_agents
        self.proxy = proxy
 
    def __random_agent(self):
        if self.user_agents and isinstance(self.user_agents, list):
            return choice(self.user_agents)
        return choice(_user_agents)
 
    def __request_url(self, url):
        try:
            response = requests.get(url, headers={'User-Agent': self.__random_agent()}, proxies={'http': self.proxy,
                                                                                                 'https': self.proxy})
            response.raise_for_status()
        except requests.HTTPError:
            raise requests.HTTPError('Received non 200 status code from Instagram')
        except requests.RequestException:
            raise requests.RequestException
        else:
            return response.text
 
    @staticmethod
    def extract_json_data(html):
        soup = BeautifulSoup(html, 'html.parser')
        body = soup.find('body')
        script_tag = body.find('script')
        raw_string = script_tag.text.strip().replace('window._sharedData =', '').replace(';', '')
        return json.loads(raw_string)
 
    def profile_page_metrics(self, profile_url):
        results = {}
        try:
            response = self.__request_url(profile_url)
            json_data = self.extract_json_data(response)
            metrics = json_data['entry_data']['ProfilePage'][0]['graphql']['user']
        except Exception as e:
            raise e
        else:
            for key, value in metrics.items():
                if key != 'edge_owner_to_timeline_media':
                    if value and isinstance(value, dict):
                        value = value['count']
                        results[key] = value
                    elif value:
                        results[key] = value
        return results
 
    def profile_page_recent_posts(self, profile_url):
        results = []
        try:
            response = self.__request_url(profile_url)
            json_data = self.extract_json_data(response)
            metrics = json_data['entry_data']['ProfilePage'][0]['graphql']['user']['edge_owner_to_timeline_media']["edges"]
            print("----->JSON")
            print(json_data)
        except Exception as e:
            raise e
        else:
            for node in metrics:
                node = node.get('node')
                if node and isinstance(node, dict):
                    results.append(node)
        return results

In [21]:
from pprint import pprint
k = InstagramScraper()
results = k.profile_page_recent_posts('https://www.instagram.com/latamairlines/?hl=en')


----->JSON
{'activity_counts': None, 'config': {'csrf_token': 'lPZWf6dOi5M5QwY4Hz0Ii0xikgeoDFbx', 'viewer': None}, 'supports_es6': True, 'country_code': 'BR', 'language_code': 'en', 'locale': 'en_US', 'entry_data': {'ProfilePage': [{'logging_page_id': 'profilePage_2957254849', 'show_suggested_profiles': False, 'graphql': {'user': {'biography': 'Queremos levar seus sonhos cada vez mais longe.\nQueremos llevar tus sueños cada vez más lejos. \nWe want to take your dreams always further.', 'blocked_by_viewer': False, 'country_block': False, 'external_url': 'http://hyperurl.co/vamoslatam', 'external_url_linkshimmed': 'https://l.instagram.com/?u=http%3A%2F%2Fhyperurl.co%2Fvamoslatam&e=ATPoqgheFfNLlQC8tpoBtM9Uhmltb-aCO6TlIqzOkTSSZuZla4nuGdsQbArvXJbqMMGq9VPuGSli_dBDxvnBWnmg', 'edge_followed_by': {'count': 829073}, 'followed_by_viewer': False, 'edge_follow': {'count': 327}, 'follows_viewer': False, 'full_name': 'LATAM Airlines', 'has_blocked_viewer': False, 'highlight_reel_count': 0, 'has_reque

In [18]:
results[0].keys()

dict_keys(['__typename', 'id', 'edge_media_to_caption', 'shortcode', 'edge_media_to_comment', 'comments_disabled', 'taken_at_timestamp', 'dimensions', 'display_url', 'edge_liked_by', 'edge_media_preview_like', 'gating_info', 'media_preview', 'owner', 'thumbnail_src', 'thumbnail_resources', 'is_video'])

In [19]:
results[0]['edge_media_to_comment']

{'count': 60}